In [1]:
# examples/train_artm.py
from torch.utils.data import DataLoader

# from artm_lib.artm.model import ARTM
from artm_lib.artm.model2 import FullEM_ARTM
from artm_lib.data.collators import ARTMCollator
from artm_lib.data.dataset import ARTMDatasetParquet
from artm_lib.preprocessing.tokenizer import simple_tokenizer
from artm_lib.preprocessing.vocabulary import build_vocab_and_index_from_parquet

import numpy as np

In [15]:
# 1. Загрузка словаря и данных
token_to_id, doc_index = build_vocab_and_index_from_parquet(
    parquet_dir_str=r".\data\parquets", tokenizer=simple_tokenizer, min_df=5
)
vocab = [""] * len(token_to_id)
for token, idx in token_to_id.items():
    vocab[idx] = token


 Scanning: input.parquet
Vocab size: 8074 | Documents: 2225



In [26]:
# 2. DataLoader
dataset = ARTMDatasetParquet(
    doc_index=doc_index,
    token_to_id=token_to_id,
    text_column="Description",
    tokenizer=simple_tokenizer,
)
loader = DataLoader(
    dataset, batch_size=1, collate_fn=ARTMCollator(len(token_to_id)), num_workers=0
)


In [ ]:
for doc_ids, bow in loader:
    print(f"doc_ids: {doc_ids}")
    print(f"Batch shape: {bow.shape}")
    print(f"Non-zero entries: {bow.nnz}")
    print(f"bow: {bow}")
    break


In [28]:
bow.shape

(1, 8074)

In [23]:
def _norm(x):
    x = np.maximum(x, np.zeros_like(x))
    norm = x.sum(axis=0)    #// суммирование по столбцам (получается одна строка)
    if norm.any() == 0:
        print(f'{norm=}')
    _eps = 1e-4
    x = np.where(norm > _eps, x / norm, np.zeros_like(x))
    return x

In [24]:
#data = X.copy()
data = DataLoader(
    dataset, batch_size=1, collate_fn=ARTMCollator(len(token_to_id)), num_workers=0
)
vocabulary = vocab.copy()

n_topics = 10
tol = 1e-3

rng = np.random.default_rng()

phi_wt = rng.uniform(size=(len(vocabulary), n_topics))

for i in range(100):
    #print(i)
    n_wt = np.zeros(shape=(len(vocabulary), n_topics))
    for doc_id, n_dw in data:
        n_dw = n_dw.todense()
        theta_td = np.full(shape=(n_topics, ), fill_value=1. / n_topics) #np.array([0.5, 0.5])
        for _ in range(100):
            n_tdw = n_dw[:, None] * _norm((phi_wt.T * theta_td[:, None]).T)
            new_theta_td = _norm(np.sum(n_tdw, axis=0))
            if np.linalg.norm(new_theta_td - theta_td) < tol:
                theta_td = new_theta_td
                break
            theta_td = new_theta_td
        n_wt += n_tdw
    new_phi_wt = _norm(n_wt)    

    diff_phi_wt = np.linalg.norm(new_phi_wt - phi_wt, 'fro')
    print(diff_phi_wt)
    if diff_phi_wt < tol:
        phi_wt = new_phi_wt
        break
    phi_wt = new_phi_wt

print(phi_wt)


ValueError: could not broadcast input array from shape (1,1,8074) into shape (1,8074,64592)